# SonarQube Measure Cleaning
The goal of this script is to clean the downloaded SonarQube data and bring the different data versions together for all the eligible projects.

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# data import
current_dir = os.getcwd()

# construct path to the project data folder
data_dir = os.path.join(current_dir, '..', '..', 'Data','Sonar_Measures')

# load SonarQube measure data
df1 = pd.read_csv(os.path.join(data_dir, 'SONAR_MEASURES_V1.csv'))
df2 = pd.read_csv(os.path.join(data_dir, 'SONAR_MEASURES_V2.csv'))

# add database source
df1['database'] = 'Version1'
df2['database'] = 'Version2'

print('--------------- VERSION 1 ---------------')
print(df1.head())
print('------------------------------------------------------------------------------------')
print('--------------- VERSION 2 ---------------')
print(df2.head())

--------------- VERSION 1 ---------------
                                 commitHash projectID        SQAnalysisDate  \
0  e0880e263e4bf8662ba3848405200473a25dfc9f  accumulo  2011-10-04T00:46:07Z   
1  e8774c5ec3a35e042f320540b5f7e66ebd2d9e87  accumulo  2011-10-04T16:57:13Z   
2  2032ebbd0ed90734da39ca238bbd10dee24d0030  accumulo  2011-10-04T18:39:18Z   
3  de297d4932e08625a5df146f0802041bb5aeb892  accumulo  2011-10-04T19:31:01Z   
4  34efaae87639a83b60fdb7274de4b45051025a3a  accumulo  2011-10-05T17:19:06Z   

   classes  files  functions  commentLines  commentLinesDensity  complexity  \
0     2108   1103      17295         13509                  6.2       43137   
1     2108   1103      17295         13507                  6.2       43137   
2     2108   1103      17295         13507                  6.2       43137   
3     2108   1103      17295         13507                  6.2       43137   
4     2108   1103      17295         13507                  6.2       43137   

   fileC

## Used variables of previous scientific work
* Number of classes
* Number of files
* Lines
* Ncloc
* Ncloc language distribution
* Number of classes and interfaces
* Missing package info
* Package
* Statements
* Number of directories
* Number of functions
* Number of comment lines
* Number of comment lines density
* Complexity
* Class complexity
* Function complexity
* Function complexity distribution
* File complexity distribution
* Cognetive complexity
* Package dependency cycles
* Coverage
* Lines to cover
* Line coverage
* Uncovered lines
* Duplicated lines
* Duplicated blocks
* Duplicated files
* Duplicated lines density

## Bringing together the two database versions
Even though the variables of the version 1 database are a subset of the variables of version 2, there are different naming conventions which makes combining the projects of both versions non-trivial. <br>
The variable names of the first version follow the following logic: exampleVariable, while the variables of version 2 look like this: EXAMPLE_VARIABLE. <br>
To bring these two dataframes together, the column names of version 1 is adapted to match the naming conventions of version 2. This is done with the goal of concatenating the two dataframes.

In [3]:
pd.options.display.max_seq_items = 2000
print(f"Version 1 column names: {df1.columns}")
print("-------------------------------------------------------------------------------------------------")
print(f"Version 2 column names: {df2.columns}")

Version 1 column names: Index(['commitHash', 'projectID', 'SQAnalysisDate', 'classes', 'files',
       'functions', 'commentLines', 'commentLinesDensity', 'complexity',
       'fileComplexity', 'classComplexity', 'functionComplexity',
       'functionComplexityDistribution', 'fileComplexityDistribution',
       'coverage', 'duplicatedLines', 'duplicatedBlocks', 'duplicatedFiles',
       'duplicatedLinesDensity', 'violations', 'blockerViolations',
       'criticalViolations', 'infoViolations', 'falsePositiveIssues',
       'confirmedIssues', 'lastCommitDate', 'codeSmells', 'bugs',
       'effortToReachMaintainabilityRatingA', 'afferentCouplings',
       'efferentCouplings', 'cognitiveComplexity', 'lines', 'ncloc',
       'nclocLanguageDistribution', 'linesToCover', 'lineCoverage',
       'majorViolations', 'minorViolations', 'openIssues', 'sqaleRating',
       'numberOfClassesAndInterfaces', 'missingPackageInfo', 'package',
       'statements', 'uncoveredLines', 'reopenedIssues', 'sqale

In [4]:
def snake_case(string):
  """Transforms a string to snake_case by inserting underscores before uppercase letters. Afterwards it transforms all letters to uppercase."""
  return re.sub(r'(?<!^)(?=[A-Z])', '_', string).upper()

snake_case_list = [snake_case(item) for item in df1.columns]
print(snake_case_list)

['COMMIT_HASH', 'PROJECT_I_D', 'S_Q_ANALYSIS_DATE', 'CLASSES', 'FILES', 'FUNCTIONS', 'COMMENT_LINES', 'COMMENT_LINES_DENSITY', 'COMPLEXITY', 'FILE_COMPLEXITY', 'CLASS_COMPLEXITY', 'FUNCTION_COMPLEXITY', 'FUNCTION_COMPLEXITY_DISTRIBUTION', 'FILE_COMPLEXITY_DISTRIBUTION', 'COVERAGE', 'DUPLICATED_LINES', 'DUPLICATED_BLOCKS', 'DUPLICATED_FILES', 'DUPLICATED_LINES_DENSITY', 'VIOLATIONS', 'BLOCKER_VIOLATIONS', 'CRITICAL_VIOLATIONS', 'INFO_VIOLATIONS', 'FALSE_POSITIVE_ISSUES', 'CONFIRMED_ISSUES', 'LAST_COMMIT_DATE', 'CODE_SMELLS', 'BUGS', 'EFFORT_TO_REACH_MAINTAINABILITY_RATING_A', 'AFFERENT_COUPLINGS', 'EFFERENT_COUPLINGS', 'COGNITIVE_COMPLEXITY', 'LINES', 'NCLOC', 'NCLOC_LANGUAGE_DISTRIBUTION', 'LINES_TO_COVER', 'LINE_COVERAGE', 'MAJOR_VIOLATIONS', 'MINOR_VIOLATIONS', 'OPEN_ISSUES', 'SQALE_RATING', 'NUMBER_OF_CLASSES_AND_INTERFACES', 'MISSING_PACKAGE_INFO', 'PACKAGE', 'STATEMENTS', 'UNCOVERED_LINES', 'REOPENED_ISSUES', 'SQALE_INDEX', 'DEVELOPMENT_COST', 'SQALE_DEBT_RATIO', 'ALERT_STATUS', '

There are some mistakes for variables that contain uppercase letters that don't mark a new word ('PROJECT_I_D', 'S_Q_ANALYSIS_DATE'). Also the variable 'database' is lowercase in the version 2 data but is currently uppercase. The variables are fixed by hand.

In [5]:
snake_case_list = ['COMMIT_HASH', 'PROJECT_ID', 'SQ_ANALYSIS_DATE', 'CLASSES', 'FILES', 'FUNCTIONS', 'COMMENT_LINES',
                   'COMMENT_LINES_DENSITY', 'COMPLEXITY', 'FILE_COMPLEXITY', 'CLASS_COMPLEXITY', 'FUNCTION_COMPLEXITY',
                   'FUNCTION_COMPLEXITY_DISTRIBUTION', 'FILE_COMPLEXITY_DISTRIBUTION', 'COVERAGE', 'DUPLICATED_LINES',
                   'DUPLICATED_BLOCKS', 'DUPLICATED_FILES', 'DUPLICATED_LINES_DENSITY', 'VIOLATIONS', 'BLOCKER_VIOLATIONS',
                   'CRITICAL_VIOLATIONS', 'INFO_VIOLATIONS', 'FALSE_POSITIVE_ISSUES', 'CONFIRMED_ISSUES', 'LAST_COMMIT_DATE', 'CODE_SMELLS',
                   'BUGS', 'EFFORT_TO_REACH_MAINTAINABILITY_RATING_A', 'AFFERENT_COUPLINGS', 'EFFERENT_COUPLINGS', 'COGNITIVE_COMPLEXITY', 'LINES',
                   'NCLOC', 'NCLOC_LANGUAGE_DISTRIBUTION', 'LINES_TO_COVER', 'LINE_COVERAGE', 'MAJOR_VIOLATIONS', 'MINOR_VIOLATIONS', 'OPEN_ISSUES',
                   'SQALE_RATING', 'NUMBER_OF_CLASSES_AND_INTERFACES', 'MISSING_PACKAGE_INFO', 'PACKAGE', 'STATEMENTS', 'UNCOVERED_LINES',
                   'REOPENED_ISSUES', 'SQALE_INDEX', 'DEVELOPMENT_COST', 'SQALE_DEBT_RATIO', 'ALERT_STATUS', 'QUALITY_GATE_DETAILS', 'QUALITY_PROFILES',
                   'NEW_SQALE_DEBT_RATIO', 'VULNERABILITIES', 'RELIABILITY_REMEDIATION_EFFORT', 'RELIABILITY_RATING', 'SECURITY_REMEDIATION_EFFORT',
                   'SECURITY_RATING', 'WONT_FIX_ISSUES', 'PACKAGE_DEPENDENCY_CYCLES', 'database']

In [6]:
df1.columns = snake_case_list
df1.head()

,COMMIT_HASH,PROJECT_ID,SQ_ANALYSIS_DATE,CLASSES,FILES,FUNCTIONS,COMMENT_LINES,COMMENT_LINES_DENSITY,COMPLEXITY,FILE_COMPLEXITY,...,QUALITY_PROFILES,NEW_SQALE_DEBT_RATIO,VULNERABILITIES,RELIABILITY_REMEDIATION_EFFORT,RELIABILITY_RATING,SECURITY_REMEDIATION_EFFORT,SECURITY_RATING,WONT_FIX_ISSUES,PACKAGE_DEPENDENCY_CYCLES,database
0,e0880e263e4bf8662ba3848405200473a25dfc9f,accumulo,2011-10-04T00:46:07Z,2108,1103,17295,13509,6.2,43137,40.6,...,"[{""key"":""css-sonar-way-41536"",""language"":""css""...",0.000000,838,7322,5,9505,4,0,0,Version1
1,e8774c5ec3a35e042f320540b5f7e66ebd2d9e87,accumulo,2011-10-04T16:57:13Z,2108,1103,17295,13507,6.2,43137,40.6,...,"[{""key"":""css-sonar-way-41536"",""language"":""css""...",0.222222,838,7081,5,9505,4,0,0,Version1
2,2032ebbd0ed90734da39ca238bbd10dee24d0030,accumulo,2011-10-04T18:39:18Z,2108,1103,17295,13507,6.2,43137,40.6,...,"[{""key"":""css-sonar-way-41536"",""language"":""css""...",0.222222,838,7081,5,9505,4,0,0,Version1
3,de297d4932e08625a5df146f0802041bb5aeb892,accumulo,2011-10-04T19:31:01Z,2108,1103,17295,13507,6.2,43137,40.6,...,"[{""key"":""css-sonar-way-41536"",""language"":""css""...",0.674560,838,7322,5,9505,4,0,0,Version1
4,34efaae87639a83b60fdb7274de4b45051025a3a,accumulo,2011-10-05T17:19:06Z,2108,1103,17295,13507,6.2,43137,40.6,...,"[{""key"":""css-sonar-way-41536"",""language"":""css""...",0.671668,838,7322,5,9505,4,0,0,Version1


In [7]:
df_conc = pd.concat([df1, df2])
df_conc.head(-50)

,COMMIT_HASH,PROJECT_ID,SQ_ANALYSIS_DATE,CLASSES,FILES,FUNCTIONS,COMMENT_LINES,COMMENT_LINES_DENSITY,COMPLEXITY,FILE_COMPLEXITY,...,sg_i.JAVA_CYCLIC_PACKAGES_PERCENT,sg_i.MAX_MODULE_NCCD,sg_i.ARCHITECTURE_FEATURE_AVAILABLE,sg_i.NUMBER_OF_ISSUES,sg_i.NUMBER_OF_CRITICAL_ISSUES_WITHOUT_RESOLUTION,sg_i.VIOLATING_COMPONENTS_PERCENT,sg_i.UNASSIGNED_COMPONENTS_PERCENT,sg_i.NUMBER_OF_THRESHOLD_VIOLATIONS,sg_i.NUMBER_OF_WORKSPACE_WARNINGS,sg_i.NUMBER_OF_IGNORED_CRITICAL_ISSUES
0,e0880e263e4bf8662ba3848405200473a25dfc9f,accumulo,2011-10-04T00:46:07Z,2108.0,1103.0,17295.0,13509.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e8774c5ec3a35e042f320540b5f7e66ebd2d9e87,accumulo,2011-10-04T16:57:13Z,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2032ebbd0ed90734da39ca238bbd10dee24d0030,accumulo,2011-10-04T18:39:18Z,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,de297d4932e08625a5df146f0802041bb5aeb892,accumulo,2011-10-04T19:31:01Z,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34efaae87639a83b60fdb7274de4b45051025a3a,accumulo,2011-10-05T17:19:06Z,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66656,NaN,org.apache:zookeeper,NaN,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66657,NaN,org.apache:zookeeper,NaN,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66658,NaN,org.apache:zookeeper,NaN,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66659,NaN,org.apache:zookeeper,NaN,748.0,434.0,4320.0,9545.0,13.2,9229.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_conc["ANALYSIS_KEY"]

0                         NaN
1                         NaN
2                         NaN
3                         NaN
4                         NaN
                 ...         
66706    AWMblPr7O48jNFNDCAof
66707    AWMbdJy4pxPbkMlK2_uE
66708    AWMbdFnspxPbkMlK29Ot
66709    AWMbdBpopxPbkMlK26Vg
66710    AWMbc8RNpxPbkMlK223T
Name: ANALYSIS_KEY, Length: 144640, dtype: object

The SQ_ANALYSIS_DATE variable is missing for all projects of the 2nd version of the Technical Debt Dataset, because it has a different database structure than the first version. The ANALYSIS_KEY variable can be connected however to the according table that contains the date of the SonarQube analysis called SONAR_ANALYSIS.

In [13]:
# read in data containing the SonarQube analysis date
analysis_df = pd.read_csv(os.path.join(data_dir, 'SONAR_ANALYSIS.csv'))

# select needed columns
analysis_df = analysis_df[["ANALYSIS_KEY", "DATE"]]

# rename columns so its naming is consistent with df_conc
analysis_df.columns = ["ANALYSIS_KEY", "SQ_ANALYSIS_DATE"]
analysis_df.head()

,ANALYSIS_KEY,SQ_ANALYSIS_DATE
0,AWedEXD3C4KKKThcCqHV,2008-07-07 14:52:05
1,AWedCu_QC4KKKThcCqGv,2008-07-07 12:31:47
2,AWedBDbIC4KKKThcCqGA,2008-07-05 18:54:27
3,AWec_b8nC4KKKThcCqFg,2008-07-03 20:21:40
4,AWec904qC4KKKThcCqE3,2008-07-02 00:12:36


In [17]:
# map the analysis keys with the according dates
analysis_key_date_map = analysis_df.set_index('ANALYSIS_KEY')['SQ_ANALYSIS_DATE'].to_dict()

# fill in missing values of the df_conc with those value pairs
for index, row in df_conc.iterrows():
    if pd.isna(row['SQ_ANALYSIS_DATE']) and row['database'] == 'Version2' and row['ANALYSIS_KEY'] in analysis_key_date_map:
        df_conc.loc[index, 'SQ_ANALYSIS_DATE'] = analysis_key_date_map[row['ANALYSIS_KEY']]

df_conc.head(-50)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,COMMIT_HASH,PROJECT_ID,SQ_ANALYSIS_DATE,CLASSES,FILES,FUNCTIONS,COMMENT_LINES,COMMENT_LINES_DENSITY,COMPLEXITY,FILE_COMPLEXITY,...,sg_i.JAVA_CYCLIC_PACKAGES_PERCENT,sg_i.MAX_MODULE_NCCD,sg_i.ARCHITECTURE_FEATURE_AVAILABLE,sg_i.NUMBER_OF_ISSUES,sg_i.NUMBER_OF_CRITICAL_ISSUES_WITHOUT_RESOLUTION,sg_i.VIOLATING_COMPONENTS_PERCENT,sg_i.UNASSIGNED_COMPONENTS_PERCENT,sg_i.NUMBER_OF_THRESHOLD_VIOLATIONS,sg_i.NUMBER_OF_WORKSPACE_WARNINGS,sg_i.NUMBER_OF_IGNORED_CRITICAL_ISSUES
0,e0880e263e4bf8662ba3848405200473a25dfc9f,accumulo,2008-07-07 14:52:05,2108.0,1103.0,17295.0,13509.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e8774c5ec3a35e042f320540b5f7e66ebd2d9e87,accumulo,2008-07-07 12:31:47,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2032ebbd0ed90734da39ca238bbd10dee24d0030,accumulo,2008-07-05 18:54:27,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,de297d4932e08625a5df146f0802041bb5aeb892,accumulo,2008-07-03 20:21:40,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34efaae87639a83b60fdb7274de4b45051025a3a,accumulo,2008-07-02 00:12:36,2108.0,1103.0,17295.0,13507.0,6.2,43137.0,40.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66656,NaN,org.apache:zookeeper,2015-03-15 00:14:39,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66657,NaN,org.apache:zookeeper,2015-03-14 22:47:06,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66658,NaN,org.apache:zookeeper,2015-03-14 22:28:06,748.0,434.0,4320.0,9545.0,13.2,9228.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66659,NaN,org.apache:zookeeper,2015-03-09 18:22:02,748.0,434.0,4320.0,9545.0,13.2,9229.0,21.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_conc.to_csv('sonar_measures_v1_v2.csv', index = False)

## Result
There now is a concatenated table of version 1 and 2 of the Technical Debt Dataset. The variable names have been aligned to match and the date of the SonarQube analysis has been added for version 2. The resulting table contains the necessary metrics and values needed to build models to predict code smells.